# Readme

README: this is a function that takes a user's username and gets email if it is in the user's bio or if it is linked as a business email.

from: https://hackernoon.com/instagram-scraper-how-to-scrape-data-from-instagram-2021-77w35h5

instagramy api: https://pypi.org/project/instagramy/

github repo: https://github.com/yogeshwaran01/instagramy

# get_user_email; function to get email from username

In [ ]:
# install necessary packages
!pip install instagramy
!pip install instagramy --upgrade

In [ ]:
from instagramy import InstagramUser
from instagramy import InstagramHashTag

In [ ]:
# Python code to find list of email addresses in a string
# Using the regular expression
import re

def find_email(string):
  lst = re.findall('\S+@\S+', string)
  return lst 

For Login into Instagram via instagramy session id is required. No username or password is needed. 

1. Login into Instagram in chrome
2. Inspect the page
3. Move to Application and then to cookies (left hand side) and copy the sessionid
*Note: Check for session id frequently, It may be changed by Instagram


In [ ]:
session_id ="12622236922%3AnyFhGw6nEufPLA%3A19"

# get business email given username
def get_user_email(username):
  user = InstagramUser(username, sessionid=session_id)
  if user.user_data['business_email'] != None:
    print(user.user_data['business_email'])
  elif '.com' in user.user_data['biography']:
    print(find_email(user.user_data['biography']))

# may want to filter by number of followers, age, and public accounts only. use user.user_data to find num of followers 
get_user_email("patricia.cnr")

KeyError: ignored

# filter_profile; function to filter profiles from username

In [ ]:
# return true or false if profile is valid
def filter_profiles(username):
  user = InstagramUser(username, sessionid=session_id)
  # if user is verified, return true
  if user.user_data['is_verified'] == True: 
    return True
  # if user has 10k+ followers and most recent post needs at least [num of followers]/5 likes, return true
  num_followers = user.user_data['edge_followed_by']['count']
  num_likes = user.user_data['edge_owner_to_timeline_media']['edges'][1]['node']['edge_liked_by']['count'] # num of likes on 2nd most recent post
  if num_followers >= 10000 and num_likes >= 500: 
    return True
  return False

# Get_profile_links
gets profile links from a given hashtag and writes to csv

In [ ]:
! pip3 install selenium

!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /Users/nicolechen/Downloads/chromedriver

import sys
sys.path.insert(0,'/Users/nicolechen/Downloads/chromedriver') # this is nicole's chromedriver
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') # this is kelsey's chromedriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException


options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

driver.get("https://www.instagram.com/accounts/login/")
print(driver.title)

username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

username.clear()
username.send_keys("campus_insights")

password.clear()
password.send_keys("IPOby2018Gm!")

Login_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']")))#.click()
driver.execute_script('arguments[0].click()', Login_button)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1

In [ ]:


searchbox = WebDriverWait(driver,50).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']"))) #error with timeout exception

searchbox.clear()
hashtag = "#food"
searchbox.send_keys(hashtag)
time.sleep(5)
searchbox.send_keys(Keys.ENTER)
time.sleep(5)
searchbox.send_keys(Keys.ENTER)
time.sleep(5)

# n_scrolls = 3
# for i in range(1, n_scrolls):
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(5)

driver.execute_script("window.scrollBy(0,1000000)")
time.sleep(5)
driver.execute_script("window.scrollBy(0,1000000)")
time.sleep(5)
driver.execute_script("window.scrollBy(0,1000000)")
time.sleep(5)

links = driver.find_elements_by_tag_name("a")

links = [link.get_attribute('href') for link in links]

profile_links = []
for link in links:
  driver.get(link)
  try:
      profile=(driver.find_element_by_tag_name("a").get_attribute('href'))
      profile_links.append(profile)
      print(profile)
  except NoSuchElementException: 
      pass

TimeoutException: ignored

# get username from profile link
outputs usernames.csv, which contains list of usernames

In [ ]:
# return username from profile link
profile_links = {x.replace("https://www.instagram.com/", "") for x in profile_links}
usernames = []
for x in profile_links: 
  if len(x) != 0 and "https" not in x: 
    x = x.rstrip(x[-1])
    usernames.append(x)

print(usernames)

emails = []
for username in usernames:
  try: 
    filter_profiles(username)
    if (filter_profiles(username) == True):
      emails.append(get_user_email(username))
  except: 
      pass

import csv
df = pd.DataFrame(emails,columns=["Emails"])
print(df)
df.to_csv('emails.csv')